## Final Assignment


Before working on this assignment please read these instructions fully. Use blackboard to submit a link to your repository. 

On blackboard your find the assessment criteria. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find at least two datasets on the web which are related, and that you build an application that visualize these datasets to answer a research question with the broad topic of **health** or **agriculture** in the **region where you were born**. The region can be a city, town or a provence.  

The research question should be a question with a causual nature. For instance questions like: How does independent variable X influence the dependent variable of Y?

The code should be programmed efficiently. Also identify the most critical part and write software test for this part. Take into account the performance of the dataprocessing

### About the data

You can merge these datasets with data from different regions if you like. For instance, you might want to compare the health effect of earhtquacks in Groningen versus Los Angelos USA. 

You are welcome to choose datasets at your discretion, but keep in mind they will be shared with others, so choose appropriate datasets. You are welcome to use datasets of your own as well, but minimual two datasets should be coming from the web and or API's. 

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations in your visualization. 

### Instructions:

Define a research question, select data and code your data acquisition, data processing, data analysis and visualization. Write code to test most critical parts. Use a repository with a commit strategy and write a readme file. 

Write a small document with the following:
- State the region and the domain category that your data sets are about 
- State the research question 
- Justify the chosen data storage and processing approach
- Justify the chosen analysis approach
- Justify the chosen data visualization approach

Upload your document and the link of your repository to black board

In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import panel as pn
import cbsodata
import bokeh
from bokeh.io import output_notebook
from bokeh.io import output_file
from bokeh.plotting import figure, show, curdoc
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Row, Column, CustomJS, DataTable, TableColumn, Whisker
from bokeh.models import HTMLTemplateFormatter
from bokeh.models import Range1d
from bokeh.models import HoverTool
from bokeh.models import Range1d
from bokeh.transform import factor_cmap
from panel.interact import interact
import yaml

pn.extension(comms='vscode')

In [180]:
# old way of obtaining data and combining it with the metadata

# def get_config():
#     """ 
#     Read in config file and return it as a dictionary. 
#     """
#     with open("C:/Users/rie12/Desktop/config.yaml", 'r') as stream:
#         config = yaml.safe_load(stream)
#         return config
    
# path = get_config()['programming1']

# crime_stat = pd.read_csv(path + '/safety_actual.csv', sep=';')
# safe_per = pd.read_csv(path + '/safety_perceiv.csv', sep=';')

# # meta data for the perceived safety file but can also be used for the crime file
# meta_data_safe = pd.read_csv(path + '/81881NED_metadata.csv', sep=';', skiprows=1)

# # getting the necessary personal characteristics for both datasets
# meta_data_safe_persoon = meta_data_safe[['ID','Title']]
# meta_data_safe_persoon = meta_data_safe_persoon[51:93]

# # merging the safety and the meta data
# safe_per = safe_per.merge(meta_data_safe_persoon, how='inner', right_on='ID',left_on='Persoonskenmerken')
# # merging the crime and meta data
# crime_stat = crime_stat.merge(meta_data_safe_persoon, how='inner', right_on='ID',left_on='Persoonskenmerken')

In [181]:
# Makes pandas show all the columns and rows.
pd.set_option('display.max_columns', None)

In [182]:
# downloading the data using a libary provided by cbs. This already has the required metadata included.
crime_stat = pd.DataFrame(cbsodata.get_data('83095NED'))
safe_per = pd.DataFrame(cbsodata.get_data('81881NED'))

# meta data, Currently not used.
# metadata = pd.DataFrame(cbsodata.get_meta('81881NED','Persoonskenmerken'))
# metadata = metadata[['Key','Title']]

<h2> Data filtering function </h2>
Function made to turn 

In [183]:
def safety_file_filter(df, df_type='safety', year='2019'):
    # obtaining only the age personal characteristics as that is what I am interested in.
    df = df[df['Persoonskenmerken'].str.contains('Leeftijd') | df['Persoonskenmerken'].str.contains('Totaal') == True]
    df = df[df['Marges'].str.contains('Waarde') == True]
    # getting only the data from the specified year
    df = df[df['Perioden'].str.contains(year) == True]
    
    # changing the data to numbers
    for columns in df:
        if columns != 'Persoonskenmerken' and columns != 'Perioden':
            df[columns] =  pd.to_numeric(df[columns], errors='coerce')
            df = df.fillna(0)
    
    # if statement to determine which columns to take & rename
    if df_type == 'safety':
        df = df[['Persoonskenmerken','VoeltZichWelEensOnveilig_1','VoeltZichVaakOnveilig_2','VanZakkenrollerij_3','VanBerovingOpStraat_4','VanInbraakInWoning_5','VanMishandeling_6']]
        
        df = df.rename(columns={'VoeltZichWelEensOnveilig_1':'Has_Felt_Unsafe', 'VoeltZichVaakOnveilig_2':'Has_Felt_Unsafe_often', 'VanZakkenrollerij_3':'Unsafe_pickpocketing',
                                'VanBerovingOpStraat_4':'Unsafe_robbery','VanInbraakInWoning_5':'Unsafe_burglary','VanMishandeling_6':'Unsafe_abuse',
                                'AantalDelicten_1':'Reported_total_crime','PogingTotInbraak_38':'Reported_burglary','Mishandeling_31':'Reported_abuse'})
        
    else:
        # Obtaining only the columns I am interested in
        df = df[['Persoonskenmerken','AantalDelicten_1', 'PogingTotZakkenrollerij_66', 'Zakkenrollerij_67', 'PogingTotBeroving_68','Beroving_69', 'PogingTotInbraak_38', 'Mishandeling_31']]
        # Adding attemted non violent street robbery and successfull non violent street robbery together.
        df['Reported_pickpocketing'] = df['PogingTotZakkenrollerij_66'] + df['Zakkenrollerij_67']
        df['Reported_robbery'] = df['PogingTotBeroving_68'] + df['Beroving_69']
        # dropping the columns that were added together.
        df = df.drop(['PogingTotZakkenrollerij_66','Zakkenrollerij_67','PogingTotBeroving_68','Beroving_69'], axis=1)
        
        df = df.rename(columns={'VoeltZichWelEensOnveilig_1':'Has_Felt_Unsafe', 'VoeltZichVaakOnveilig_2':'Has_Felt_Unsafe_often', 'VanZakkenrollerij_3':'Unsafe_pickpocketing',
                                'VanBerovingOpStraat_4':'Unsafe_robbery','VanInbraakInWoning_5':'Unsafe_burglary','VanMishandeling_6':'Unsafe_abuse',
                                'AantalDelicten_1':'Reported_total_crime','PogingTotInbraak_38':'Reported_burglary','Mishandeling_31':'Reported_abuse'})
    
    df = df.head(1).append(df.tail(7))
    
    return df


In [184]:
# Applying the function and merging the result.
# 2018 data not working at the moment/.
year_list = ['2012','2013','2014', '2015','2016','2017','2019']
safe = {}
crime = {}
both = {}
for year in year_list:
    crime[year] = safety_file_filter(crime_stat, df_type='crime', year=year)
    safe[year] = safety_file_filter(safe_per, df_type='safety', year=year)
    
    both[year] = crime[year].merge(safe[year], how='inner', left_on='Persoonskenmerken', right_on='Persoonskenmerken')

<h1> Plotting graphs </h1>

In [185]:
pairings = {'Total':        ['Has_Felt_Unsafe', 'Reported_total_crime'],
            'Pickpocketing':['Unsafe_pickpocketing', 'Reported_pickpocketing'],
            'Robbery':      ['Unsafe_robbery', 'Reported_robbery'],
            'Burglary':     ['Unsafe_burglary', 'Reported_burglary'],
            'Abuse':        ['Unsafe_abuse', 'Reported_abuse']
            }

In [209]:
def interact_bar(df_key,key):
       
    merge_df = both[df_key]
    
    fig = px.bar(merge_df, x='Persoonskenmerken', y=pairings[key], title='Unsafety vs Reported crime per age group in ' + df_key, barmode='group',
                 labels={
                     "Persoonskenmerken": "Age groups",
                     "value": "% of total population",
                     "variable": ""
                 })
    
    fig.update_layout({
    'template': 'plotly_dark',
    #'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    
    
    return fig

In [210]:
def interact_barbelplot(df_key,key):

    merge_df = both[df_key]

    fig = px.scatter(merge_df, x=pairings[key], y=merge_df['Persoonskenmerken'], labels={'variable':''})
    fig.update_layout(title_text = 'Unsafety vs Reported crime per age group in ' + df_key,
        xaxis_title = "% of total population", yaxis_title = 'Age groups')
    
    
    for i in range(merge_df.shape[0]):
        fig.add_shape(
            type='line',
            x0=merge_df[pairings[key][0]].iloc[i], y0=merge_df.index[i], 
            x1=merge_df[pairings[key][1]].iloc[i], y1=merge_df.index[i],
            line_color="#88FF00")
        
    if key == 'Total':
        fig.update_xaxes(range=[0, 60])
    elif key == 'Burglary':
        fig.update_xaxes(range=[0, 18])
    else:
        fig.update_xaxes(range=[-1, 9])
        
    # Changing the layout
    fig.update_layout({
    'template': 'plotly_dark',
    #'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    
    # Giving the markers a green shading.
    fig.update_traces(marker=dict(size=15,
                            line=dict(width=1,
                                    color='#88FF00')),
                selector=dict(mode='markers'))
    
    
    
    return fig

In [211]:
pairing_selection = pn.widgets.RadioButtonGroup(options = ['Total','Pickpocketing','Robbery','Burglary','Abuse'], button_type = 'success')
year_selection = pn.widgets.Select(name='Select', options=['2012', '2013', '2014', '2015', '2016', '2017', '2019'],value='2019')

interactive_barchart = pn.interact(interact_bar, df_key = year_selection ,key = pairing_selection)
interactive_barbell = pn.interact(interact_barbelplot, df_key = year_selection, key = pairing_selection)

In [212]:
pn.serve(pn.Column(interactive_barbell,interactive_barchart[1]))

Launching server at http://localhost:51626
